In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

In [4]:
CLIENT_ID = 'PJYR2NYMP0T1Q2ET1KTYPPK43UCGBB5KDENW11W4ONC1PD4W' # your Foursquare ID
CLIENT_SECRET = 'LBTT0JZEZYHX3I3M5DNLQDLZZOFQBDKMPJBTLVZWFDQDJCUD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: PJYR2NYMP0T1Q2ET1KTYPPK43UCGBB5KDENW11W4ONC1PD4W


The Foursquare API gives us the nearest 100 stadium in the city.


In [40]:
LIMIT = 500 
cities = ["New York, NY", 'Chicago, IL', 'San Francisco, CA', 'Jersey City, NJ', 'Boston, MA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d189941735") 
    results[city] = requests.get(url).json()

In [41]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [43]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of Football Stadium in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top Stadiums")

Total number of Football Stadium in New York, NY =  146
Showing Top Stadiums
Total number of Football Stadium in Chicago, IL =  77
Showing Top Stadiums
Total number of Football Stadium in San Francisco, CA =  47
Showing Top Stadiums
Total number of Football Stadium in Jersey City, NJ =  41
Showing Top Stadiums
Total number of Football Stadium in Boston, MA =  29
Showing Top Stadiums


In [45]:
maps[cities[0]]

In [46]:
maps[cities[1]]

In [47]:
maps[cities[2]]

In [48]:
maps[cities[3]]

In [49]:
maps[cities[4]]

In [50]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.041069871448912554
Chicago, IL
Mean Distance from Mean coordinates
0.05319101745396471
San Francisco, CA
Mean Distance from Mean coordinates
0.02909977090345476
Jersey City, NJ
Mean Distance from Mean coordinates
0.06375630269774274
Boston, MA
Mean Distance from Mean coordinates
0.143874792611992


In [ ]:
We see , San francisco and New york are the most dense cities with more football stadium. 

To get concrete measure of this density. we will get the  mean location of the footballstadium which should be near to most of them if they are really dense or far if not.
Next I will take the average of the distance of the venues to the mean coordinates.

In [56]:
maps[cities[0]]

In [57]:
maps[cities[1]]

In [58]:
maps[cities[2]]

In [59]:
maps[cities[3]]

In [60]:
maps[cities[4]]

We now see that Sanfransisco is his best option for our tourist's to enjoy lot of football matchs and visit more football stadium and  book a hotel near that mean coordinate to surround himself , and enjoy his trip in USA, watching more matches in sanfransico without travelling to many cities.

Let's have our tourist to enjoy his time in USA, especiaally in Sanfrancisco.